# 17 Introduction to Simple Linear Regression 

In [3]:
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## What is Linear Regression?
**Linear regression** is a statistical method used to model the relationship between a dependent variable (target) and one or more independent variables (features). It is widely used for prediction and forecasting in various fields such as economics, finance, and science.

Assuming that there is a linear relationship between the independent variable(s) and the dependent variable and that this relationship can be represented by a straight line (or hyperplane), it aims to find the best-fitting linear equation that describes this relationship. 

### Applications:
- **Predicting Exam Scores:** Imagine you have data on students' study hours and their corresponding exam scores. You can use linear regression to predict a student's exam score based on the number of hours they studied.

- **Forecasting House Prices:** Suppose you have data on house sizes (in square feet) and their selling prices. You can use linear regression to predict the selling price of a house based on its size.

- **Estimating Gas Mileage:** If you have data on cars' engine sizes and their corresponding gas mileage, you can use linear regression to predict a car's gas mileage based on its engine size.

## Theory Behind Linear Regression

Recall the equation for a straight line from your early math classes,

$$ y = mx + b$$

The equation represents a straight line where $m$ is the slope and $b$ is the y-intercept.

Here's a Python code example using `matplotlib` to plot the line represented by the equation $ y = mx + b$ and allowing you to adjust the values of $m$ and $b$.

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider, Checkbox

# Function to plot the line
def plot_line(m, b):
    x_vals = np.linspace(0, 10, 100)
    y_vals = m * x_vals + b
    plt.figure(figsize=(5, 4))
    plt.plot(x_vals, y_vals, color='red', label=f'y = {m:.2f}x + {b:.2f}')
    plt.xlabel('X')
    plt.ylabel('y')
    plt.title('y = mx + b')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 10)
    plt.ylim(-20, 20)

# Define sliders for m and b
m_slider = FloatSlider(min=-10, max=10, step=1, value=0, description='Slope (m)')
b_slider = FloatSlider(min=-10, max=10, step=1, value=0, description='Intercept (b)')

# Create interactive plot
interact(plot_line, m=m_slider, b=b_slider)

<Figure size 500x400 with 0 Axes>

interactive(children=(IntSlider(value=0, description='m', max=1), IntSlider(value=0, description='b', max=1), …

<function __main__.plot_line(m, b)>

Now imagine we have data on house sizes (in square feet) and their selling prices. Below, we generate some synthetic data for house sizes and selling prices.

In [ ]:
# Generate some random data for house sizes and selling prices
np.random.seed(0)
X = np.random.randint(1000, 3000, 50)  # House sizes in square feet
y = 50 * X + np.random.normal(0, 10000, 50)  # Selling prices

# Create a DataFrame
df = pd.DataFrame({'House Size (sqft)': X, 'Selling Price': y})

# Display the DataFrame
df

Now let's plot these data points in a scatter plot.

In [ ]:
# Function to plot the data and multiple lines with different fits
def plot_data_and_lines(show_lines, show_legend):
    plt.figure(figsize=(9, 5))
    plt.scatter(X, y, color='blue', label='Data')

    # Plot multiple lines with different fits
    if show_lines:
        for i in range(25, 100, 25):  # Generate 5 lines with different fits
            m = i
            b = np.mean(y) - m * np.mean(X)  # Calculate intercept
            x_vals = np.linspace(min(X), max(X), 100)
            y_vals = m * x_vals + b
            plt.plot(x_vals, y_vals, label=f'$\hat{{y}} = {m:.2f}x + {b:.2f}$')

    # Show legend if specified
    if show_legend:
        plt.legend()

    plt.xlabel('House Size (sqft)\n$(x)$')
    plt.ylabel('Selling Price\n$(y)$')
    plt.title('House Size vs Selling Price')
    plt.grid(True)
    plt.show()

# Checkbox widget to show/hide the lines
lines_checkbox = Checkbox(value=False, description='Show Lines')  # Default value set to False

# Checkbox widget to show/hide the legend
legend_checkbox = Checkbox(value=False, description='Show Legend')  # Default value set to False

# Create interactive plot with checkboxes
interact(plot_data_and_lines, show_lines=lines_checkbox, show_legend=legend_checkbox)

Let's imagine 3 data scientists are working with the same dataset. If each scientist draws a different line of fit, how do they decide which line is best?

How can we find a simple linear equation that best represents the relationship between the dependent variable, *Selling Price*, and the independent variable, *House Size (sqft)*? In other words, how do we find the **line of best fit**?

### Line of Best Fit:
The line of best fit represents the linear relationship between the independent variable (predictor) and the dependent variable (response). In our case, the independent variable is *House Size (sqft)* and the dependent variable is *Selling Price*. 

This line is often determined through linear regression, which aims to minimize **the difference between the observed values and the values predicted by the line**.

### What are Residuals?
Residuals, denoted as $ε$ (epsilon), are the differences between the observed values ($y$) and the values predicted by the model ($\hat{y}$). In other words, they represent the error in the model's predictions. Mathematically, residuals can be expressed as,

$$ε_{i} = y_{i} - \hat{y}_{i}$$
$$~~~~~~~~~~~~~~~~~~= y_{i} - (mx_{i} + b)$$

where:
- $ε_{i}$ is the error or residual for the $i$-th data point

- $y_{i}$ is the observed (actual) value for the $i$-th data point

- $\hat{y}_{i}$ is the predicted value by the model for the $i$-th data point

- $m$ represents the slope of the line in a linear regression model

- $x_{i}$ represents the value of the independent variable for the $i$-th data point

- $b$ represents the y-intercept of the line in a linear regression model

A residual is a measure of how well a line fits an individual data point. Consider this simple data set with a line of fit drawn through it.

<p align="center">
  <img src="imgs/residual1.png" alt="Alt text" width="400" height="400">
</p>

and notice how point **(2, 8)** is **<span style="color:green">4</span>** units above the line:

<p align="center">
  <img src="imgs/residual2.png" alt="Alt text" width="400" height="400">
</p>

This vertical distance is known as a **residual**. For data points above the line, the residual is positive, and for data points below the line, the residual is negative.

For example, the residual for the point **(4, 3)** is **<span style="color:red">-2</span>**.

<p align="center">
  <img src="imgs/residual3.png" alt="Alt text" width="400" height="400">
</p>

The closer a data point's residual is to 0 the better the fit. In this case, the line fits the point (4, 3) better than (2, 8).

### Visualizing Residuals
We can further explore residuals by visualizing how they relate to our linear regression model for our housing dataset. Here, we can adjust the slope ($m$) and intercept ($b$) of the regression line and observe the corresponding residuals.

In [ ]:
# Function to plot the data, line, and residuals
def plot_data_line_residuals(m, b, show_line, show_residuals):
    plt.figure(figsize=(9, 5))
    plt.scatter(X, y, color='blue', label='Data')

    # Calculate predicted prices using the selected m and b
    y_hat = m * X + b
    
    # Plot the regression line if show_line is True
    if show_line:
        plt.plot(X, y_hat, color='red', label=f'$\hat{{y}} = {m}x + {b}$')

    # Plot dashed lines representing residuals if show_residuals is True
    if show_residuals:
        for i in range(len(X)):
            plt.plot([X[i], X[i]], [y[i], y_hat[i]], color='green', linestyle='--', linewidth=0.8)

        # Add legend for residuals if not already added
        handles, labels = plt.gca().get_legend_handles_labels()
        if 'Residuals' not in labels:
            plt.plot([], [], color='green', linestyle='--', label='Residuals')

    plt.xlabel('House Size (sqft)\n$(x)$')
    plt.ylabel('Selling Price\n$(y)$')
    plt.title('House Size vs Selling Price')
    plt.legend()
    plt.grid(True)
    plt.show()

# Define sliders for m and b
m_slider = FloatSlider(min=-100, max=100, step=1, value=0, description='Slope (m)')
b_slider = FloatSlider(min=-50000, max=50000, step=1000, value=0, description='Intercept (b)')

# Checkbox widget to show/hide the line
line_checkbox = Checkbox(value=False, description='Show Line')  # Default value set to False

# Checkbox widget to show/hide the residuals
residuals_checkbox = Checkbox(value=False, description='Show Residuals')  # Default value set to False

# Create interactive plot with sliders and checkboxes
interact(plot_data_line_residuals, show_line=line_checkbox, show_residuals=residuals_checkbox, m=m_slider, b=b_slider)


So how do we know we've found the model parameters ($m$ and $b$) for the **line of best fit**?

### Determining Model Parameters for the Line of Best Fit
#### Method of Least Squares
In linear regression, the model parameters for the line of best fit are determined using the **method of least squares**. This method aims to <u>minimize</u> the sum of the squared differences between the observed values and the values predicted by the regression line.

#### Mathematical Formulation
Given a set of $n$ data points $(x_{i}, y_{i})$, where $x_{i}$ represents the independent variable and $y_{i}$ represents the corresponding dependent variable, the line of best fit is represented by the equation:

$$ \hat{y}_{i} = mx_{i} + b$$

where:
- $\hat{y}_{i}$ is the predicted value by the model for the $i$-th data point

- $m$ is the slope of the line (coefficient for the independent variable $x$)

- $x_{i}$ represents the value of the independent variable for the $i$-th data point

- $b$ is the y-intercept of the line

The goal is to find the values of $m$ and $b$ that minimize the **sum of the squared errors**, denoted as $SSE$:

$$SSE = \sum_{i=1}^{n}ε_{i}^2 = \sum_{i=1}^{n}(y_{i} - \hat{y}_{i})^2 = \sum_{i=1}^{n}(y_{i} - (mx_{i} + b))^2$$

where:
- $\hat{y}_{i}$ is the predicted value of $y_{i}$ at the $i$-th data point

With our same housing dataset, we can adjust the slope ($m$) and intercept ($b$) of the regression line and observe the corresponding sum of squared errors.

In [ ]:
# Function to plot the data, line, residuals, and SSE
def plot_data_line_residuals_sse(m, b, show_line, show_residuals):
    plt.figure(figsize=(9, 5))
    plt.scatter(X, y, color='blue', label='Data')

    # Calculate predicted prices using the selected m and b
    y_hat = m * X + b
    
    # Calculate residuals
    residuals = y - y_hat
    
    # Calculate sum of squared residuals if the line is shown
    if show_line:
        sum_squared_residuals = np.sum(residuals**2)
    else:
        sum_squared_residuals = None
    
    # Plot the regression line if show_line is True
    if show_line:
        plt.plot(X, y_hat, color='red', label=f'$\hat{{y}} = {m}x + {b}$')

    # Plot dashed lines representing residuals if show_residuals is True
    if show_residuals:
        for i in range(len(X)):
            plt.plot([X[i], X[i]], [y[i], y_hat[i]], color='green', linestyle='--', linewidth=0.8)

        # Add legend for residuals if not already added
        handles, labels = plt.gca().get_legend_handles_labels()
        if 'Residuals' not in labels:
            plt.plot([], [], color='green', linestyle='--', label='Residuals')

    plt.xlabel('House Size (sqft)\n$(x)$')
    plt.ylabel('Selling Price\n$(y)$')
    title = 'House Size vs Selling Price'
    if show_line:
        title += f'\nSum of Squared Errors: {sum_squared_residuals:.2e}'
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

# Define sliders for m and b
m_slider = FloatSlider(min=-100, max=100, step=1, value=0, description='Slope (m)')
b_slider = FloatSlider(min=-50000, max=50000, step=1000, value=0, description='Intercept (b)')

# Checkbox widget to show/hide the line
line_checkbox = Checkbox(value=False, description='Show Line')  # Default value set to False

# Checkbox widget to show/hide the residuals
residuals_checkbox = Checkbox(value=False, description='Show Residuals')  # Default value set to False

# Create interactive plot with sliders and checkboxes
interact(plot_data_line_residuals_sse, show_line=line_checkbox, show_residuals=residuals_checkbox, m=m_slider, b=b_slider)


#### Introduction to Cost Function
Now that we've discussed the method of least squares and the goal of minimizing the sum of squared errors ($SSE$) to find the coefficients for the line of best fit, let's delve deeper into the concept of the **cost function**.

In the realm of machine learning and optimization algorithms, a **cost function** serves as a critical component in evaluating the performance of a model. Also known as a loss function or objective function, it quantifies how well the model's predictions align with the actual observed values in the training dataset.

##### Purpose of Cost Function
The primary purpose of a cost function is twofold:

1. **Evaluation of Model Performance**: By assessing the extent of error or deviation between the predicted and actual values, the cost function provides insights into the efficacy of the model in capturing the underlying patterns and relationships within the data. A lower cost indicates better alignment between predictions and observations, signifying higher model accuracy.

2. **Optimization**: Beyond evaluation, the cost function plays a pivotal role in the optimization process, guiding the iterative adjustment of model parameters to minimize prediction errors. Optimization algorithms, such as gradient descent, leverage the gradient (partial derivatives) of the cost function with respect to the model parameters to iteratively update the parameters and converge towards the optimal solution. More on this later...

#### Cost Function: Sum of Squared Errors ($SSE$)
As we've seen earlier, the sum of squared errors ($SSE$) serves as a measure of the discrepancy between the observed values and the values predicted by our regression line. While $SSE$ is effective in quantifying the overall error, it has some limitations.

#### Limitations of $SSE$
Although $SSE$ provides valuable insight into the model's performance, it does not account for the number of data points in the dataset. As a result, $SSE$ may vary significantly depending on the size of the dataset, making it challenging to compare models trained on different datasets directly.

#### Introducing Mean Squared Error ($MSE$)
To address the limitations of $SSE$, we introduce the concept of **Mean Squared Error** ($MSE$). $MSE$ is obtained by dividing the $SSE$ by the number of data points, resulting in the average squared error per data point. Mathematically, it is written as:

$$ MSE = \frac{1}{n}SSE = \frac{1}{n}\sum_{i=1}^{n}ε_{i}^2 = \frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat{y}_{i})^2 = \frac{1}{n}\sum_{i=1}^{n}(y_{i} - (mx_{i} + b))^2$$

This normalization ensures that the cost function is independent of the dataset size, ensuring that the loss function is consistent across datasets of varying sizes. Additionally, $MSE$ provides a more intuitive measure of the model's performance, representing the average squared difference between the predicted and actual values.

With our same housing dataset, we can adjust the slope ($m$) and intercept ($b$) of the regression line and observe the corresponding mean squared error ($MSE$).

In [ ]:
# Function to plot the data, line, residuals, and MSE
def plot_data_line_residuals_mse(m, b, show_line, show_residuals):
    plt.figure(figsize=(9, 5))
    plt.scatter(X, y, color='blue', label='Data')

    # Calculate predicted prices using the selected m and b
    y_hat = m * X + b
    
    # Calculate residuals
    residuals = y - y_hat
    n = len(residuals)
    
    # Calculate mean sequared error if the line is shown
    if show_line:
        mean_squared_error = (1/n)*np.sum(residuals**2)
    else:
        mean_squared_error = None
    
    # Plot the regression line if show_line is True
    if show_line:
        plt.plot(X, y_hat, color='red', label=f'$\hat{{y}} = {m}x + {b}$')

    # Plot dashed lines representing residuals if show_residuals is True
    if show_residuals:
        for i in range(len(X)):
            plt.plot([X[i], X[i]], [y[i], y_hat[i]], color='green', linestyle='--', linewidth=0.8)

        # Add legend for residuals if not already added
        handles, labels = plt.gca().get_legend_handles_labels()
        if 'Residuals' not in labels:
            plt.plot([], [], color='green', linestyle='--', label='Residuals')

    plt.xlabel('House Size (sqft)\n$x$')
    plt.ylabel('Selling Price\n$y$')
    title = 'House Size vs Selling Price'
    if show_line:
        title += f'\nMean Squared Error: {mean_squared_error:.2e}'
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

# Define sliders for m and b
m_slider = FloatSlider(min=-100, max=100, step=1, value=0, description='Slope (m)')
b_slider = FloatSlider(min=-50000, max=50000, step=1000, value=0, description='Intercept (b)')

# Checkbox widget to show/hide the line
line_checkbox = Checkbox(value=False, description='Show Line')  # Default value set to False

# Checkbox widget to show/hide the residuals
residuals_checkbox = Checkbox(value=False, description='Show Residuals')  # Default value set to False

# Create interactive plot with sliders and checkboxes
interact(plot_data_line_residuals_mse, show_line=line_checkbox, show_residuals=residuals_checkbox, m=m_slider, b=b_slider)

#### Gradient Descent
Now that we've established $MSE$ as our preferred cost function, let's explore how we can optimize our linear regression model using **gradient descent**. Gradient descent is an iterative optimization algorithm that aims to <u>minimize</u> the cost function ($MSE$) by adjusting the model parameters (slope and intercept).

##### Mathematics Behind Gradient Descent 
Up to this point, we've been using $\hat{y}_{i}$ to represent the predicted value for the $i$-th training example,

$$ \hat{y}_{i} = mx_{i} + b$$

where:
- $\hat{y}_{i}$ is the predicted value for the $i$-th data point

- $m$ is the slope of the line (coefficient for the independent variable $x$)

- $x_{i}$ represents the value of the independent variable for the $i$-th data point

- $b$ is the y-intercept of the line

We can also express the linear regression with the following notation,

$$ \hat{y}_{i} = w_{0} + w_{1}x_{i}$$

where:

- $\hat{y}_{i}$ is the predicted value for the $i$-th data point

- $w_{0}$ corresponds to the y-intercept ($b$)

- $w_{1}$ corresponds to the slope ($m$)

- $x_{i}$ represents the value of the independent variable for the $i$-th data point

To minimize the cost function ($MSE$), the model needs to find the best value of $w_{0}$ and $w_{1}$. We will find the optimal values for $w_{0}$ and $w_{1}$ using gradient descent in a step-by-step process.

##### Step-by-Step Process of Gradient Descent
1. **Initialization**: We initialize the values of $w_{0}$ and $w_{1}$ to some random values or zeros. These values represent the parameters (y-intercept and slope) of the linear regression model.

2. **Define the Cost Function:** We define the Mean Squared Error ($MSE$) as our cost function. The $MSE$ represents the average squared difference between the predicted and actual values over all data points

$$ MSE = \frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat{y}_{i})^2$$

We can actually scale our cost function using a factor of $\frac{1}{2n}$ so that we get a cost function that looks like,

$$ J(w) = \frac{1}{2n}\sum_{i=1}^{n}(\hat{y}_{i} - y_{i})^2$$

The factor $\frac{1}{2}$ is often included for mathematical convenience as it simplifies the derivative computation in the next step. 

The subtraction $\hat{y}_{i} - y_{i}$ is just a rearrangement; it serves the same purpose of quantifying the discrepancy between predicted and actual values, ultimately resulting in the same optimization goal of minimizing the cost function. For example, $(3 - 1)^2 = 2^2 = 4$ which is the same as  $(1 - 3)^2 = (-2)^2 = 4$

3. **Compute the Gradient:** Compute the gradient of the cost function with respect to each parameter. It involves taking the partial differentiation of the cost function with respect to each of the parameters.

Partial derivative with respect to $w_0$ simplifies to,

$$\frac{\partial J(w)}{\partial w_0} = \frac{1}{n}\sum_{i=1}^{n}(\hat{y}_{i} - y_{i})$$ 

Partial derivative with respect to $w_1$ simplifies to,

$$\frac{\partial J(w)}{\partial w_1} = \frac{1}{n}\sum_{i=1}^{n}(\hat{y}_{i} - y_{i}) \cdot x_{i}$$ 

4. **Set the Learning Rate:** We choose a learning rate, denoted as $\alpha$, which determines the size of the steps we take in the direction of the gradient.

5. **Update the Parameters:** Using the gradient and the learning rate, we update the parameters iteratively. The update rule for each parameter is:

$$w_{0} = w_{0} - \alpha \cdot \frac{\partial J(w)}{\partial w_0}$$

$$w_{1} = w_{1} - \alpha \cdot \frac{\partial J(w)}{\partial w_1}$$

The generalized update rule for each model parameter is

$$\text{new coefficient} = \text{old coefficient} - (\text{learning rate} * \text{gradient})$$

We apply this update rule to each parameter, moving them in the direction that reduces the cost function.

6. **Repeat Until Convergence:** We repeat steps 3 through 5 until the cost function converges to a minimum. Convergence is typically checked by monitoring the change in the cost function or after a predetermined number of iterations.

7. **Final Parameters:** Once the algorithm converges, the final parameters $w_0$ and $w_1$ represent the best-fit line for our linear regression model.

Lets try to visualize this process by plotting our cost function ($MSE$) against one of our model parameter's, specifically $w_1$ which represents the slope of our regression line. $w_1$ also represents the weight or coefficient of the parameter *House Size (sqft)*.

In [ ]:
# Define the cost function
def cost_function(w0, w1, x, y):
    y_hat = w0 + w1 * x
    errors = y_hat - y
    n = len(x)
    cost = (1 / (2 * n)) * np.sum(errors ** 2)
    return cost

# Derivative of the cost function with respect to theta1 (slope)
def w1_partial_derivative(w0, w1, x, y):
    y_hat = w0 + w1 * x
    errors = y_hat - y
    n = len(x)
    derivative = (1 / n) * np.sum(errors * x) 
    return derivative

# Interactive function to plot the cost function against a specified weight
def plot_cost_function(weight_value, show_tangent_line):
    # Create a range of weight values for plotting
    weights = np.linspace(-1000, 1000, 100)
    
    # Calculate the cost for each weight value
    costs = [cost_function(0, w, X, y) for w in weights]
    
    # Plot the cost function
    plt.figure(figsize=(9, 5))
    plt.plot(weights, costs, label='Cost Function')
    
    # Plot the specified weight value as a green point
    plt.plot(weight_value, cost_function(0, weight_value, X, y), 'o', markersize=8, color='black')
    
    if show_tangent_line:
        # Calculate the slope of the tangent line (derivative of the cost function)
        slope = w1_partial_derivative(0, weight_value, X, y)
        
        # Plot the tangent line
        tangent_x = np.linspace(weight_value - 300, weight_value + 300, 100)
        tangent_y = slope * (tangent_x - weight_value) + cost_function(0, weight_value, X, y)
        plt.plot(tangent_x, tangent_y, 'r--', label='Tangent Line')
    
    plt.xlabel('$w_1$')
    plt.ylabel('MSE')
    plt.title('Cost Function (MSE) vs $w_1$')
    plt.grid(True)
    plt.legend()
    plt.show()

# Define sliders for weight value and tangent line visibility
weight_slider = FloatSlider(min=-1000, max=1000, step=10, value=-700, description='w_1')
tangent_line_checkbox = Checkbox(value=False, description='Show Tangent Line')

# Create interactive plot
interact(plot_cost_function, weight_value=weight_slider, show_tangent_line=tangent_line_checkbox)

In the plot above, The cost function quantifies how well our model fits the training data for different values of $w_1$.

Now, let's focus on a specific point on the cost function curve. At that point, the tangent line touches the curve, indicating the slope of the curve at that precise location. The gradient of the cost function at this point gives us the slope of this tangent line. 

This gradient essentially tells us how the cost function will change and in what direction if we adjust $w_1$ slightly.

**If the slope of the tangent line is +positive**: $~~w_j = w_j - \alpha~\cdot~$ (+ve value). Hence the value of $w_j$ decreases.

<p align="center">
  <img src="imgs/gradient_descent1.png" alt="Alt text" width="400" height="400">
</p>

**If the slope of the tangent line is -negative**: $~~w_j = w_j - \alpha~\cdot~$ (-ve value). Hence the value of $w_j$ increases.

<p align="center">
  <img src="imgs/gradient_descent2.png" alt="Alt text" width="400" height="400">
</p>

Again, let's visualize the cost function ($MSE$) against the parameter $w_1$ which represents the slope of the regression line. As a reminder, the goal of gradient descent is to find the optimal value of $w_1$ that minimizes the cost function.

In [ ]:
# Define the cost function
def cost_function(w0, w1, x, y):
    predictions = w0 + w1 * x
    errors = predictions - y
    n = len(x)
    cost = (1 / (2 * n)) * np.sum(errors ** 2)
    return cost

# Compute the gradient of the cost function
def compute_gradient(w0, w1, x, y):
    predictions = w0 + w1 * x
    errors = predictions - y
    n = len(x)
    gradient_w0 = (1 / n) * np.sum(errors)
    gradient_w1 = (1 / n) * np.sum(errors * x)
    return gradient_w0, gradient_w1

# Interactive function to plot the cost function and the steps to the minimum
def plot_steps(w1, show_steps):
    w0 = 0
    alpha = 0.0000001 
    num_iterations = 20     

    # Create a range of weight values for plotting
    weights = np.linspace(-1000, 1000, 100)
    
    # Calculate the cost for each weight value
    costs = [cost_function(0, w, X, y) for w in weights]
    
    # Plot the cost function
    plt.figure(figsize=(9, 5))
    plt.plot(weights, costs, label='Cost Function')
    
    # Plot the starting point
    cost = cost_function(w0, w1, X, y)
    plt.plot(w1, cost, 'o', markersize=8, color="black")
    # print(f"Iteration 0: w_0 = {w0}, w_1 = {w1}, Cost = {cost}, Alpha: {alpha}")

    if show_steps:
        if w1 >= 52.74:
            rad = -0.3
        else:
            rad = 0.3

        # Perform gradient descent
        for i in range(num_iterations):
            # Compute the gradient
            gradient_w0, gradient_w1 = compute_gradient(w0, w1, X, y)
            
            # Update the parameters
            prev_w0, prev_w1 = w0, w1
            w0 -= alpha * gradient_w0
            w1 -= alpha * gradient_w1
            
            # Compute the cost
            cost = cost_function(w0, w1, X, y)
            
            # Display the updated parameters and cost
            plt.plot(w1, cost, 'o', markersize=8, color="black")
            
            # Add arrow indicating the direction of movement
            plt.annotate('', xy=(w1, cost), xytext=(prev_w1, cost_function(prev_w0, prev_w1, X, y)),
                         arrowprops=dict(arrowstyle='->', connectionstyle=f'arc3,rad={rad}', lw=1.5, color="red"))
            # print(f"Iteration {i + 1}: w_0 = {w0}, w_1 = {w1}, Cost = {cost}, Alpha: {alpha}")
        
    
    plt.xlabel('$w_1$')
    plt.ylabel('MSE')
    plt.title('Cost Function (MSE) vs $w_1$')
    plt.grid(True)
    plt.legend()
    plt.show()


# Define sliders for initial weight and learning rate, and a checkbox for showing steps/path
w1_slider = FloatSlider(min=-1000, max=1000, step=10, value=-700, description='w_1')
steps_checkbox = Checkbox(value=False, description='Show Steps')

# Create interactive plot
interact(plot_steps, w1=w1_slider, show_steps=steps_checkbox)

In the plot above, we can set the initial point which corresponds to to the initial chosen value of $w_1$. 

If the "Show Steps" checkbox is selected, the red arrows illustrate the iterative steps taken by the gradient descent algorithm to minimize the cost function. At each step, the algorithm computes the gradient of the cost function with respect to $w_1$ and updates $w_1$ (and $w_0$) accordingly. 

These updates happen iteratively until the cost function converges to a minimum or after a set number of iterations.

#### How To Choose Learning Rate 
The choice of the learning rate, $\alpha$, is very important as it ensures that Gradient Descent converges in a reasonable time.

If we choose **$\alpha$ to be very large**, Gradient Descent can overshoot the minimum. It may fail to converge or even diverge.

<p align="center">
  <img src="imgs/alpha1.png" alt="Alt text" width="300" height="300">
</p>

If we choose **$\alpha$ to be very small**, Gradient Descent will take small steps to reach local minima and will take a longer time to reach minima. 

<p align="center">
  <img src="imgs/alpha2.png" alt="Alt text" width="300" height="300">
</p>

### Linear Regression with `scikit-learn `
While implementing gradient descent from scratch offers deep insights into optimization techniques, we can leverage existing libraries like `scikit-learn` to streamline the process. `scikit-learn`  encapsulates complex optimization algorithms, such as gradient descent variants, within its user-friendly interface, allowing us to focus on model development rather than algorithmic intricacies.

Here, we instantiate a Linear Regression model object and then train it using the fit method with our dataset. Behind the scenes, `scikit-learn` employs efficient optimization algorithms, including variants of gradient descent, to find the optimal parameters that minimize the difference between predicted and actual target values.

In [ ]:
# Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X for fitting
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

# Fit the model on the training data
model = LinearRegression()
model.fit(X_train, y_train)

### Interpreting the Final Model Parameters
Having trained our Linear Regression model using `scikit-learn`, we've successfully optimized its parameters to best fit the data. Let's examine the final model parameters:

In [ ]:
print(f"Intercept: {model.intercept_}")
print(f"Coefficients: {model.coef_}")

With these values, we can write the equation of our final regression model,

$$\hat{y} = w_{0} + w_{1}x = -6744.28 + 52.52x$$

where:

- $\hat{y}$ is the predicted (*Selling Price*) value 

- $w_{0}$ corresponds to the y-intercept ($b$)

- $w_{1}$ corresponds to the slope ($m$)

- $x$ represents the value of the independent variable (*House Size (sqft)*)

The **intercept** represents the model's prediction when all features are zero. It accounts for the baseline value of the target variable, independent of the input features.

On the other hand, the **coefficients** (also known as weights or slopes) quantify the impact of each feature on the target variable. The coefficient represents the change in the target variable for a one-unit change in the input feature, while holding all other features constant.

For example, we can say that we expect the price to increase by $52.52 for every additional square foot of house size, with all other factors held constant.

Let's plot our final regression model:

In [ ]:
# Making predictions on entire dataset
y_hat = model.intercept_ + model.coef_[0]*X
# y_hat = model.predict(X)

# Plotting the data points
plt.scatter(X_train, y_train, color='black', label='Training Set')
plt.scatter(X_test, y_test, color='blue', label='Test Set')

# Plotting the regression line
plt.plot(X, y_hat, color='red', label=f'$\hat{{y}} = {round(model.intercept_, 2)} + {round(model.coef_[0], 2)}x$ ')

# Adding labels and title
plt.xlabel('House Size (sqft)\n$(x)$')
plt.ylabel('Selling Price\n$(y)$')
plt.title('Linear Regression Model')
plt.legend()

# Displaying the plot
plt.show()

### Model Performance Metrics
After building a regression model, it's crucial to assess its performance using appropriate metrics. These metrics help quantify how well the model fits the data and how accurate its predictions are.

#### Mean Absolute Error (MAE)

Mean Absolute Error (MAE) measures the average absolute difference between the predicted values and the actual values. It provides a straightforward interpretation of the model's predictive accuracy.

Mathematically, MAE is calculated as:

$$ MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i| $$

where:
- $ n $ is the number of samples
- $ y_i $ is the actual target value for the $ i $-th sample
- $ \hat{y}_i $ is the predicted target value for the $ i $-th sample

#### Mean Squared Error (MSE)

Mean Squared Error (MSE) measures the average squared difference between the predicted values and the actual values. It penalizes larger errors more heavily than smaller ones.

Mathematically, MSE is calculated as:

$$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

#### Root Mean Squared Error (RMSE)

Root Mean Squared Error (RMSE) is the square root of the Mean Squared Error. It is often preferred over MSE because it is in the same units as the target variable, making it more interpretable.

Mathematically, RMSE is calculated as:

$$ RMSE = \sqrt{MSE} $$

#### R-squared ($ R^2 $) Score

R-squared ($ R^2 $) score, also known as the coefficient of determination, measures the proportion of the variance in the target variable that is predictable from the independent variables. It provides an indication of the goodness of fit of the model.

The $ R^2 $ score ranges from 0 to 1, where:
- 0 indicates that the model does not explain any variability in the target variable
- 1 indicates that the model perfectly explains the variability in the target variable

The $ R^2 $ score is calculated as:

$$ R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2} $$

where:
- $ \bar{y} $ is the mean of the actual target values

When interpreting the $ R^2 $ score, it's essential to compare it with the $ R^2 $ score of a baseline model (e.g., a model that always predicts the mean of the target variable). A higher $ R^2 $ score indicates that the model performs better than the baseline model.

#### Implementing Model Performance Metrics with `scikit-learn`

To calculate these performance metrics using `scikit-learn`, we can use the appropriate functions provided by the `metrics` module. For example:

In [ ]:
# Make predictions on the test data
y_hat = model.predict(X_test)

# Create a DataFrame for X_test and y_test
df_test = pd.DataFrame({
    'House Size (sqft)': X_test.flatten(), 
    'Selling Price': y_test, 
    'Predictions': y_hat},
    )
df_test

When evaluating a linear regression model, it's important to assess its performance on both the training set and the test set to evaluate for underfitting/overfitting. For just this lesson, we are evaluating model metrics for just the test set.

In [ ]:
# Calculate MAE
mae = metrics.mean_absolute_error(y_test, y_hat)

# Calculate MSE
# mse = metrics.mean_squared_error(y_test, y_hat)

# Calculate RMSE
# rmse = np.sqrt(mse)

# Calculate R-squared score
r_squared = metrics.r2_score(y_test, y_hat)

print(f"Mean Absolute Error (MAE): {mae}")
# print(f"Mean Squared Error (MSE): {mse}")
# print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared Score (R^2): {r_squared}")

With the calculated RMSE, we can tell our stakeholders that our model's predictions are off by ~$8,594.24 on average.

### Making Predictions
We can use our linear regression model to make predictions. Lets say that some developers built a 2,000 sqft house and they need help settling on a selling price. Given the size, we can use our model to predict the selling price.

In [ ]:
# X_new is your new data for which you want to make predictions
X_new = np.array(2000).reshape(1, -1)

# Make predictions on new data
predicted_selling_price = model.predict(X_new)

# Print the predictions
print(f"We can advise the developers to list the house for: ${round(predicted_selling_price[0], 2)}")

We made the prediction using the `scikit-learn` library. Let's calculate by hand to confirm. Remember our final model,
$$\hat{y} = w_{0} + w_{1}x = -6744.28 + 52.52x$$

In [ ]:
print(f"w0 = {model.intercept_}")
print(f"w1 = {model.coef_[0]}")
print(f"x = 2000")
print(f"y = w0 + w1*x = {round(model.intercept_, 2)} + {round(model.coef_[0], 2)}*2000 = {round(model.intercept_ + model.coef_[0] * 2000, 2)}")

We can also see where this prediction lies in our regression line,

In [ ]:
# Making predictions on entire dataset
y_hat = model.intercept_ + model.coef_[0]*X
# y_hat = model.predict(X)

# Plotting the data points
plt.scatter(X, y, color='black', label='Data Points')

# Plotting the regression line
plt.plot(X, y_hat, color='red', label=f'$\hat{{y}} = {round(model.intercept_, 2)} + {round(model.coef_[0], 2)}x$ ')

# Plotting the predicted value
plt.scatter(X_new, predicted_selling_price, color='green', marker='*', s=200, zorder=5, label='Predicted Value')

# Adding labels and title
plt.xlabel('House Size (sqft)\n$(x)$')
plt.ylabel('Selling Price\n$(y)$')
plt.title('Linear Regression Model')
plt.legend()

# Displaying the plot
plt.show()